# --- Day 2: Rock Paper Scissors ---

The Elves begin to set up camp on the beach. To decide whose tent gets to be closest to the snack storage, a giant Rock Paper Scissors tournament is already in progress.

Rock Paper Scissors is a game between two players. Each game contains many rounds; in each round, the players each simultaneously choose one of Rock, Paper, or Scissors using a hand shape. Then, a winner for that round is selected: Rock defeats Scissors, Scissors defeats Paper, and Paper defeats Rock. If both players choose the same shape, the round instead ends in a draw.

Appreciative of your help yesterday, one Elf gives you an encrypted strategy guide (your puzzle input) that they say will be sure to help you win. "The first column is what your opponent is going to play: A for Rock, B for Paper, and C for Scissors. The second column--" Suddenly, the Elf is called away to help with someone's tent.

The second column, you reason, must be what you should play in response: X for Rock, Y for Paper, and Z for Scissors. Winning every time would be suspicious, so the responses must have been carefully chosen.

The winner of the whole tournament is the player with the highest score. Your total score is the sum of your scores for each round. The score for a single round is the score for the shape you selected (1 for Rock, 2 for Paper, and 3 for Scissors) plus the score for the outcome of the round (0 if you lost, 3 if the round was a draw, and 6 if you won).

Since you can't be sure if the Elf is trying to help you or trick you, you should calculate the score you would get if you were to follow the strategy guide.

For example, suppose you were given the following strategy guide:
```
A Y
B X
C Z
```
This strategy guide predicts and recommends the following:

In the first round, your opponent will choose Rock (A), and you should choose Paper (Y). This ends in a win for you with a score of 8 (2 because you chose Paper + 6 because you won).
In the second round, your opponent will choose Paper (B), and you should choose Rock (X). This ends in a loss for you with a score of 1 (1 + 0).
The third round is a draw with both players choosing Scissors, giving you a score of 3 + 3 = 6.
In this example, if you were to follow the strategy guide, you would get a total score of 15 (8 + 1 + 6).

What would your total score be if everything goes exactly according to your strategy guide?

In [ ]:
import logging
import sys

sys.path.append("..")
from aoc import get_logger, get_data, timer, test

logger = get_logger("day02", logging.INFO)
data = get_data("day02")

logger.info("initialization complete!")

## Solution Part One

In [ ]:
def parsed(input_: str) -> list:
    return list(map(lambda x: x.split(), input_.splitlines()))

We'll write a function to play RPS, which calculates who wins.

In [ ]:
# making it more human-readable
ROCK, PAPER, SCISSORS = 1, 2, 3
LOSE, DRAW, WIN = 0, 3, 6

scorebook = {
    "A": (ROCK, "rock"),
    "B": (PAPER, "paper"),
    "C": (SCISSORS, "scissors"),
    "X": (ROCK, "rock"),
    "Y": (PAPER, "paper"),
    "Z": (SCISSORS, "scissors"),
}


def play(opp_hand: str, hand: str) -> (int, int):
    opp_score, score = scorebook[opp_hand][0], scorebook[hand][0]
    if score == opp_score:
        logger.debug(f"Your {scorebook[hand][1]} draws to the elf's {scorebook[opp_hand][1]}.")
        return (DRAW, score)
    elif (
        (opp_score == ROCK and score == PAPER)
        or (opp_score == PAPER and score == SCISSORS)
        or (opp_score == SCISSORS and score == ROCK)
    ):
        logger.debug(f"Your {scorebook[hand][1]} wins to the elf's {scorebook[opp_hand][1]}.")
        return (WIN, score)
    else:
        logger.debug(f"Your {scorebook[hand][1]} loses to the elf's {scorebook[opp_hand][1]}.")
        return (LOSE, score)

We can now try out our RPS function, with random input for our opponent.
You can try it out by answering the prompt with A, B or C for rock, paper or scissor.
If you leave it empty it will pick your hand at random as well.

In [ ]:
import random

try:
    user_hand = input("Rock, paper, scissor or random? [A/B/C]")
except Exception as e:  # default to random choice if user input is not supported
    logger.warning(e)
    user_hand = random.choice(["A", "B", "C"])
user_hand = user_hand.upper() if user_hand.upper() in ["A", "B", "C"] else random.choice(["A", "B", "C"])

score = sum(play(random.choice(["A", "B", "C"]), user_hand))
logger.info(f"Your score is {score}")

In [ ]:
@timer
def part1(input_):
    if type(input_) == str:
        input_ = parsed(input_)
    return sum([sum(play(p[0], p[1])) for p in input_])


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 1:", part1(data.input))

# --- Part Two ---

The Elf finishes helping with the tent and sneaks back over to you. "Anyway, the second column says how the round needs to end: X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win. Good luck!"

The total score is still calculated in the same way, but now you need to figure out what shape to choose so the round ends as indicated. The example above now goes like this:

In the first round, your opponent will choose Rock (A), and you need the round to end in a draw (Y), so you also choose Rock. This gives you a score of 1 + 3 = 4.
In the second round, your opponent will choose Paper (B), and you choose Rock so you lose (X) with a score of 1 + 0 = 1.
In the third round, you will defeat your opponent's Scissors with Rock for a score of 1 + 6 = 7.
Now that you're correctly decrypting the ultra top secret strategy guide, you would get a total score of 12.

Following the Elf's instructions for the second column, what would your total score be if everything goes exactly according to your strategy guide?

## Solution Part Two
I' don't feel like changing my RPS function, so I'm just going to adjust the input to make it work with the function from part 1.

In [ ]:
added_explanation = {"X": LOSE, "Y": DRAW, "Z": WIN}


def convert(input_) -> [str]:
    adapted_input = []
    for p in input_:
        for h in ["A", "B", "C"]:  # we'll bruteforce it
            if play(opp_hand=p[0], hand=h)[0] == added_explanation[p[1]]:
                adapted_input.append((p[0], h))
                break
    return adapted_input


@timer
def part2(input_):
    adapted_input = convert(parsed(input_))
    return part1(adapted_input)


test(func=part2, input_=data.example_input, expected=data.test_answer2)

print("solution part 2:", part2(data.input))

# Performance tuning
Just for fun, I'll try to speed up the code a bit.

In [ ]:
outcomes = {
    (ROCK, ROCK): DRAW,
    (ROCK, PAPER): WIN,
    (ROCK, SCISSORS): LOSE,
    (PAPER, ROCK): LOSE,
    (PAPER, PAPER): DRAW,
    (PAPER, SCISSORS): WIN,
    (SCISSORS, ROCK): WIN,
    (SCISSORS, PAPER): LOSE,
    (SCISSORS, SCISSORS): DRAW,
}


@timer
def part1(input_):
    score = 0
    for p in parsed(input_):
        hand = scorebook[p[1]][0]
        opp_hand = scorebook[p[0]][0]
        score += outcomes[(opp_hand, hand)] + hand
    return score


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 1:", part1(data.input))

In [ ]:
outcomes_alt = {
    (ROCK, DRAW): ROCK,
    (ROCK, WIN): PAPER,
    (ROCK, LOSE): SCISSORS,
    (PAPER, DRAW): PAPER,
    (PAPER, LOSE): ROCK,
    (PAPER, WIN): SCISSORS,
    (SCISSORS, DRAW): SCISSORS,
    (SCISSORS, WIN): ROCK,
    (SCISSORS, LOSE): PAPER,
}


@timer
def part2(input_):
    score = 0
    for p in parsed(input_):
        outcome = added_explanation[p[1]]
        opp_hand = scorebook[p[0]][0]
        hand = outcomes_alt[(opp_hand, outcome)]
        score += outcome + hand
    return score


test(func=part2, input_=data.example_input, expected=data.test_answer2)

print("solution part 2:", part2(data.input))